In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score

In [2]:
train_fd=pd.read_csv("/content/sample_data/train.csv")
test_fd=pd.read_csv("/content/sample_data/test.csv")

In [3]:
train_fd.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,WeightCategory
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [4]:
test_fd.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,15533,Female,19.007177,1.772449,137.852618,yes,yes,3.000000,3.000000,Sometimes,no,2.007348,no,1.465931,0.813235,Sometimes,Public_Transportation
1,15534,Female,21.572114,1.698346,75.000000,yes,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.577824,1.865851,Sometimes,Public_Transportation
2,15535,Male,22.285024,1.737453,82.000000,yes,yes,2.000000,2.720642,Sometimes,no,1.830614,no,0.451009,0.000000,Sometimes,Public_Transportation
3,15536,Male,30.916426,1.775580,120.860386,yes,yes,2.712747,3.000000,Sometimes,no,2.144368,no,0.712726,0.100048,Sometimes,Automobile
4,15537,Female,18.000000,1.670000,65.000000,no,yes,2.000000,3.000000,Sometimes,no,2.000000,no,1.000000,1.000000,no,Public_Transportation


In [5]:
X_train = train_fd.iloc[:, 1:17]
y_train = train_fd.iloc[:, 17]

X_test=test_fd.iloc[:,1:17]

In [6]:
categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC',
                    'SMOKE', 'SCC', 'CALC', 'MTRANS']

In [7]:
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder()
y_train = target_encoder.fit_transform(y_train)

le = LabelEncoder()
for col in categorical_cols:
    X_train[col] = le.fit_transform(X_train[col])

In [8]:
for col in categorical_cols:
    X_test[col] = le.fit_transform(X_test[col])

In [9]:
X_test

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,0,19.007177,1.772449,137.852618,1,1,3.000000,3.000000,2,0,2.007348,0,1.465931,0.813235,1,3
1,0,21.572114,1.698346,75.000000,1,1,2.000000,3.000000,2,0,2.000000,0,1.577824,1.865851,1,3
2,1,22.285024,1.737453,82.000000,1,1,2.000000,2.720642,2,0,1.830614,0,0.451009,0.000000,1,3
3,1,30.916426,1.775580,120.860386,1,1,2.712747,3.000000,2,0,2.144368,0,0.712726,0.100048,1,0
4,0,18.000000,1.670000,65.000000,0,1,2.000000,3.000000,2,0,2.000000,0,1.000000,1.000000,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5220,1,25.137087,1.766626,114.187096,1,1,2.919584,3.000000,2,0,2.151809,0,1.330519,0.196680,1,3
5221,1,18.000000,1.710000,50.000000,0,1,3.000000,4.000000,1,0,1.000000,0,2.000000,1.000000,1,3
5222,1,20.101026,1.819557,105.580491,1,1,2.407817,3.000000,2,0,2.000000,0,1.158040,1.198439,2,3
5223,1,33.852953,1.700000,83.520113,1,1,2.671238,1.971472,2,0,2.144838,0,0.000000,0.973834,2,0


In [10]:
X_train.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,1,24.443011,1.699998,81.669950,1,1,2.000000,2.983297,2,0,2.763573,0,0.000000,0.976473,1,3
1,0,18.000000,1.560000,57.000000,1,1,2.000000,3.000000,1,0,2.000000,0,1.000000,1.000000,2,0
2,0,18.000000,1.711460,50.165754,1,1,1.880534,1.411685,2,0,1.910378,0,0.866045,1.673584,2,3
3,0,20.952737,1.710730,131.274851,1,1,3.000000,3.000000,2,0,1.674061,0,1.467863,0.780199,1,3
4,1,31.641081,1.914186,93.798055,1,1,2.679664,1.971472,2,0,1.979848,0,1.967973,0.931721,1,3


In [11]:
import numpy as np

num_classes = len(np.unique(y_train))
print("Number of classes:", num_classes)

Number of classes: 7


# **1.Random Forest 🌴**

In [13]:
rf_random.fit(X_train, y_train)
print("Best Parameters:", rf_random.best_params_)
print("Best CV Accuracy:", rf_random.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Parameters: {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
Best CV Accuracy: 0.8992476328818191


In [14]:
best_model = rf_random.best_estimator_
print("Best Hyperparameters:", rf_random.best_params_)

Best Hyperparameters: {'n_estimators': 600, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}


In [15]:
best_model.fit(
    X_train, y_train,
)

RandomForestClassifier(bootstrap=False, max_features='log2',
                       min_samples_split=10, n_estimators=600, random_state=42)

In [16]:
y_test_bm = best_model.predict(X_test)

In [17]:
y_test_bm

array([4, 5, 6, ..., 2, 6, 3])

In [18]:
predicted_labels = target_encoder.inverse_transform(y_test_bm)

In [19]:
predicted_labels

array(['Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II',
       ..., 'Obesity_Type_I', 'Overweight_Level_II', 'Obesity_Type_II'],
      dtype=object)

In [20]:
output_df = pd.DataFrame({
    'id': test_fd['id'] if 'id' in test_fd.columns else range(1, len(test_fd)+1),
    'Predicted_WeightCategory': predicted_labels
})


In [21]:
output_df.to_csv('predictions_RF.csv', index=False)
print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv


**Random Forest gave us and accuracy of about 89.476% when checked with the test_y result in kaggle and 89.96% in final**

# **2.AdaBoost  🌠**

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
base_model = DecisionTreeClassifier(max_depth=1)
ada_model = AdaBoostClassifier(
    estimator=base_model,
    n_estimators=100,
    learning_rate=0.8,
    random_state=42
)

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5, 1.0]
}

grid = GridSearchCV(estimator=ada, param_grid=param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best CV Accuracy:", grid.best_score_)

best_model = grid.best_estimator_


Best Parameters: {'learning_rate': 0.5, 'n_estimators': 150}
Best CV Accuracy: 0.7235569889647434


In [30]:
y_test_bm2 = best_model.predict(X_test)

In [31]:
predicted_labels2 = target_encoder.inverse_transform(y_test_bm2)

In [32]:
predicted_labels2

array(['Obesity_Type_III', 'Normal_Weight', 'Overweight_Level_II', ...,
       'Obesity_Type_I', 'Overweight_Level_II', 'Obesity_Type_III'],
      dtype=object)

In [33]:
output_df2 = pd.DataFrame({
    'id': test_fd['id'] if 'id' in test_fd.columns else range(1, len(test_fd)+1),
    'Predicted_WeightCategory': predicted_labels2
})


In [34]:
output_df.to_csv('predictions_ADA.csv', index=False)
print("Predictions saved to predictions_ADA.csv")

Predictions saved to predictions_ADA.csv


**Ada Boost gave Accuracy of 87% in prediction in kaggle and 89% in final**

# **3. XGBoost WO HyperTune 🎷 (Random Search_CV)**

In [35]:
xgb = XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=num_classes,
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)
param_grid = {
    'n_estimators': [800, 1000, 1200, 1500],
    'learning_rate': [0.01, 0.02, 0.03, 0.05],
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 2, 3, 5],
    'gamma': [0, 0.1, 0.3, 0.5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.01, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2, 3]
}
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=20,
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [36]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:50:56] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_con...
                                           num_class=7, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9],
                                        'gamma': [0, 0.1, 0.3, 0.5],
                                        'learning_rate': [0.01, 0.02, 0.03,
                                                          0.05],
                                        'max_depth': [3, 4, 5, 6],
                                        'min_child_weight': [1, 2, 3, 5],
                                        'n_estimators': [800, 1000, 1200, 1500],
                                        'reg_alpha': [0, 0.01, 0.1, 0.5],
                                        'reg_lambda': [1, 1.5, 2, 3],
                                        'subsample': [0.7, 0.8, 0.9]},
                   random_state=42, scoring='accuracy', verbose=2)

In [37]:
best_model3 = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

Best Hyperparameters: {'subsample': 0.7, 'reg_lambda': 3, 'reg_alpha': 0.01, 'n_estimators': 800, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.02, 'gamma': 0.1, 'colsample_bytree': 0.8}


In [45]:
best_model3.fit(
    X_train, y_train,
    verbose=True
)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [19:54:34] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=0.1,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=800, n_jobs=-1, num_class=7, ...)

In [46]:
y_test_bm = best_model3.predict(X_test)

In [47]:
y_test_bm

array([4, 5, 6, ..., 2, 6, 3])

In [48]:
predicted_labels = target_encoder.inverse_transform(y_test_bm)

In [49]:
predicted_labels

array(['Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II',
       ..., 'Obesity_Type_I', 'Overweight_Level_II', 'Obesity_Type_II'],
      dtype=object)

In [50]:
output_df = pd.DataFrame({
    'id': test_fd['id'] if 'id' in test_fd.columns else range(1, len(test_fd)+1),
    'Predicted_WeightCategory': predicted_labels
})


In [51]:
output_df.to_csv('predictions_XG_WO_Tuned.csv', index=False)
print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv


**Gave Accuracy of 91.1% in kaggle and 91.7% in final result**

# **4.XG_Boost_Tuned 💯 (Optuna)**

In [53]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 8.4 MB/s eta 0:00:00


In [55]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 450, 550),
        'learning_rate': trial.suggest_float('learning_rate', 0.04, 0.07),
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3),
        'gamma': trial.suggest_float('gamma', 0.30, 0.36),
        'subsample': trial.suggest_float('subsample', 0.75, 0.82),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.48, 0.54),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.45, 0.60),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.05, 0.15),

        # fixed values
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': num_classes,
        'random_state': 42,
        'n_jobs': -1
    }

    model = XGBClassifier(**params)
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy', n_jobs=-1)
    return np.mean(scores)

# Run the new Optuna study
study = optuna.create_study(direction='maximize', study_name="XGB_Optuna_FineTuned")
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Show best results
print("Best Trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Best Hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Train final model with tuned parameters
best_params = trial.params
best_xgb = XGBClassifier(
    **best_params,
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=num_classes,
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1
)
best_xgb.fit(X_train, y_train)


[I 2025-11-03 20:08:38,438] A new study created in memory with name: XGB_Optuna_FineTuned


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-03 20:08:53,686] Trial 0 finished with value: 0.9076806578223957 and parameters: {'n_estimators': 457, 'learning_rate': 0.045095832534849806, 'max_depth': 5, 'min_child_weight': 3, 'gamma': 0.3596712936269341, 'subsample': 0.7782341983154785, 'colsample_bytree': 0.4984097972085053, 'reg_alpha': 0.46740746449708614, 'reg_lambda': 0.059171560827707116}. Best is trial 0 with value: 0.9076806578223957.
[I 2025-11-03 20:09:04,922] Trial 1 finished with value: 0.905105437214881 and parameters: {'n_estimators': 516, 'learning_rate': 0.04494286971171331, 'max_depth': 3, 'min_child_weight': 3, 'gamma': 0.3109816866289266, 'subsample': 0.7587257717368039, 'colsample_bytree': 0.49287152441878795, 'reg_alpha': 0.5206035887408256, 'reg_lambda': 0.13039396373463852}. Best is trial 0 with value: 0.9076806578223957.
[I 2025-11-03 20:09:15,843] Trial 2 finished with value: 0.9089039678153754 and parameters: {'n_estimators': 459, 'learning_rate': 0.059578366151721136, 'max_depth': 4, 'min_chi

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [20:18:25] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.48032493913982177, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, feature_weights=None,
              gamma=0.34585488120607116, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.059578366151721136,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=459, n_jobs=-1, num_class=7, ...)

**Best Parameters for the same
Best hyperparameters: {'n_estimators': 473, 'learning_rate': 0.056175675016670416, 'max_depth': 4, 'min_child_weight': 2, 'gamma': 0.3354617624351886, 'subsample': 0.7871797069127701, 'colsample_bytree': 0.5071468557602662, 'reg_alpha': 0.5393018307093757, 'reg_lambda': 0.0809287158560023} Best accuracy:0.91294**



In [ ]:
y_test_bm = model.predict(X_test)
y_test_bm
predicted_labels = target_encoder.inverse_transform(y_test_bm)
predicted_labels
output_df = pd.DataFrame({
    'id': test_fd['id'] if 'id' in test_fd.columns else range(1, len(test_fd)+1),
    'Predicted_WeightCategory': predicted_labels
})
output_df.to_csv('predictions_Tuned.csv', index=False)
print("Predictions saved to predictions.csv")

# **Final tuned Model gave accuracy of 91.37% in kaggle and 92.037% in final results**